In [1]:
import pandas as pd
from datetime import date, timedelta
import numpy as np
import matplotlib.pyplot as plt
import warnings
import tensorflow
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [25]:
df = pd.read_csv('data_cleaned.csv')

In [26]:
df

,height_x,weight_x,reach_x,fight_time_sum_x,kds_per_min_x,sub_att_per_min_x,pass_per_min_x,rev_per_min_x,tds_acc_x,tds_succ_per_min_x,tds_att_per_min_x,tot_str_acc_x,tot_str_succ_per_min_x,tot_str_att_per_min_x,sig_str_head_acc_x,sig_str_head_succ_per_min_x,sig_str_head_att_per_min_x,sig_str_body_acc_x,sig_str_body_succ_per_min_x,sig_str_body_att_per_min_x,sig_str_leg_acc_x,sig_str_leg_succ_per_min_x,sig_str_leg_att_per_min_x,sig_str_dist_acc_x,sig_str_dist_succ_per_min_x,sig_str_dist_att_per_min_x,sig_str_clinch_acc_x,sig_str_clinch_succ_per_min_x,sig_str_clinch_att_per_min_x,sig_str_ground_acc_x,sig_str_ground_succ_per_min_x,sig_str_ground_att_per_min_x,age_x,weightclass_Bantamweight_x,weightclass_Catch Weight_x,weightclass_Featherweight_x,weightclass_Flyweight_x,weightclass_Heavyweight_x,weightclass_Light Heavyweight_x,weightclass_Lightweight_x,weightclass_Middleweight_x,weightclass_Open Weight_x,weightclass_Strawweight_x,weightclass_Super Heavyweight_x,weightclass_Welterweight_x,stance_Open Stance_x,stance_Orthodox_x,stance_Sideways_x,stance_Southpaw_x,stance_Switch_x,height_y,weight_y,reach_y,fight_time_sum_y,kds_per_min_y,sub_att_per_min_y,pass_per_min_y,rev_per_min_y,tds_acc_y,tds_succ_per_min_y,tds_att_per_min_y,tot_str_acc_y,tot_str_succ_per_min_y,tot_str_att_per_min_y,sig_str_head_acc_y,sig_str_head_succ_per_min_y,sig_str_head_att_per_min_y,sig_str_body_acc_y,sig_str_body_succ_per_min_y,sig_str_body_att_per_min_y,sig_str_leg_acc_y,sig_str_leg_succ_per_min_y,sig_str_leg_att_per_min_y,sig_str_dist_acc_y,sig_str_dist_succ_per_min_y,sig_str_dist_att_per_min_y,sig_str_clinch_acc_y,sig_str_clinch_succ_per_min_y,sig_str_clinch_att_per_min_y,sig_str_ground_acc_y,sig_str_ground_succ_per_min_y,sig_str_ground_att_per_min_y,age_y,weightclass_Bantamweight_y,weightclass_Catch Weight_y,weightclass_Featherweight_y,weightclass_Flyweight_y,weightclass_Heavyweight_y,weightclass_Light Heavyweight_y,weightclass_Lightweight_y,weightclass_Middleweight_y,weightclass_Open Weight_y,weightclass_Strawweight_y,weightclass_Super Heavyweight_y,weightclass_Welterweight_y,stance_Open Stance_y,stance_Orthodox_y,stance_Sideways_y,stance_Southpaw_y,stance_Switch_y,fx_win
0,1.75,135.0,1.78,15.00,0.00,0.07,0.07,0.07,0.00,0.00,0.00,0.80,7.27,9.13,0.22,0.33,1.53,0.86,0.80,0.93,0.33,0.07,0.20,0.35,0.60,1.73,0.50,0.20,0.40,0.75,0.40,0.53,25.05,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1.78,135.0,1.78,2.78,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.26,2.52,9.71,0.10,0.72,7.55,0.50,0.36,0.72,1.00,1.44,1.44,0.20,1.80,8.99,1.00,0.72,0.72,0.00,0.00,0.00,27.63,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1.73,155.0,1.75,15.00,0.00,0.07,0.07,0.00,0.00,0.00,0.20,0.55,6.20,11.33,0.22,1.20,5.53,0.90,1.20,1.33,0.70,1.27,1.80,0.37,2.07,5.53,0.51,1.60,3.13,0.00,0.00,0.00,25.10,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1.70,155.0,1.78,9.72,0.00,0.31,0.10,0.00,0.50,0.10,0.21,0.48,2.26,4.73,0.32,0.93,2.88,0.50,0.31,0.62,0.50,0.10,0.21,0.33,0.93,2.78,0.38,0.31,0.82,1.00,0.10,0.10,29.69,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2,1.73,155.0,1.75,20.00,0.00,0.05,0.10,0.00,0.29,0.10,0.35,0.54,5.50,10.10,0.20,1.00,4.90,0.88,1.10,1.25,0.73,1.10,1.50,0.33,1.60,4.80,0.55,1.55,2.80,1.00,0.05,0.05,27.93,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1.70,155.0,1.75,89.80,0.00,0.01,0.07,0.02,0.25,0.06,0.22,0.44,5.39,12.35,0.26,2.23,8.51,0.57,0.14,0.26,0.82,0.81,0.99,0.27,2.05,7.47,0.58,0.50,0.87,0.45,0.63,1.41,31.81,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
3,1.73,155.0,1.75,35.73,0.00,0.03,0.06,0.00,0.38,0.08,0.22,0.52,6.91,13.21,0.30,2.32,7.70,0.83,1.23,1.48,0.70,1.09,1.57,0.37,2.94,8.00,0.61,1.46,2.38,0.69,0.25,0.36,28.66,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1.75,155.0,1.78,0.73,1.37,0.00,0.00,0.00,0.00,0.00,0.00,0.42,6.85,16.44,0.40,5.48,13.70,1.00,1.37,1.37,0.00,0.00,1.37,0.33,2.74,8.22,0.00,0.00,1.37,0.60,4.11,6.85,24.69,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
4,1.73,155.0,1.75,60.36,0.00,0.02,0.05,0.00,0.33,0.08,0.25,0.54,6.96,13.01,0.27,1.94,7.07,0.80,1.14,1.42,0.68,0.75,1.09,0.31,2.14,6.88,0.60,1.38,2.29,0.73,0.31,0.43,29.47,0,0,0,0,0,

In [27]:
cat_cols = ['weightclass_Bantamweight_x','weightclass_Catch Weight_x','weightclass_Featherweight_x',
          'weightclass_Flyweight_x','weightclass_Heavyweight_x','weightclass_Light Heavyweight_x',
          'weightclass_Lightweight_x','weightclass_Middleweight_x','weightclass_Open Weight_x',
          'weightclass_Strawweight_x','weightclass_Super Heavyweight_x','weightclass_Welterweight_x',
          'weightclass_Bantamweight_y','weightclass_Catch Weight_y','weightclass_Featherweight_y',
          'weightclass_Flyweight_y','weightclass_Heavyweight_y','weightclass_Light Heavyweight_y',
          'weightclass_Lightweight_y','weightclass_Middleweight_y','weightclass_Open Weight_y',
          'weightclass_Strawweight_y','weightclass_Super Heavyweight_y','weightclass_Welterweight_y',
          'stance_Open Stance_x','stance_Orthodox_x','stance_Sideways_x','stance_Southpaw_x','stance_Switch_x',
          'stance_Open Stance_y','stance_Orthodox_y','stance_Sideways_y','stance_Southpaw_y','stance_Switch_y']

In [28]:
df_num = df.drop(columns=cat_cols)
df_num.drop(columns=['fx_win'], inplace = True)
num_cols = df_num.columns

In [29]:
len(df.columns)

101

In [30]:
y = df[['fx_win']]

In [31]:
X = df.drop(columns=['fx_win'])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=700, random_state=50)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=50)

In [33]:
# col_tran = ColumnTransformer(
#             [("standard", StandardScaler(), num_cols)],
#                 remainder='passthrough')

In [34]:
# X_train_s = pd.DataFrame(col_tran.fit_transform(X_train))
# X_val_s = pd.DataFrame(col_tran.transform(X_val))
# X_test_s = pd.DataFrame(col_tran.transform(X_test))

# X_train_s = col_tran.fit_transform(X_train)
# X_val_s = col_tran.transform(X_val)
# X_test_s = col_tran.transform(X_test)

In [35]:
# y_train = y_train.values
# y_val = y_val.values
# y_test = y_test.values

In [36]:
log_reg = LogisticRegression()
log_reg.fit(X_train_s, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
score = log_reg.score(X_train_s, y_train)
score

0.6264044943820225

In [55]:
rand_forest = RandomForestClassifier(n_estimators=1000, n_jobs=-1, min_samples_leaf=100, max_depth=20)
rand_forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=100, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [57]:
rand_forest.score(X_val, y_val)

0.5409090909090909